# Date Exploratory (EDA)

Use this notebook to explore the data generated by the pipeline in your preferred programming language.

**Note**: This notebook is not executed as part of the pipeline.

## Import Libraries

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, TimestampType, FloatType
import pyspark.sql.functions as F

## 1. Bronze Layer Inspection

### 1.1 Load Bronze Table

In [0]:
catalog_name = 'workspace.capstone_project'
df_bronze = spark.table(f'{catalog_name}.tfs_incidents_bronze')
display(df_bronze.limit(10))

In [0]:
df_bronze.printSchema()

In [0]:
print("Rows:", df_bronze.count())

### 1.2 Basic Profiling Bronze

#### 1.2.1 Check Missing Value 

In [0]:
# Column-level missingness
n = df_bronze.count()
missing = df_bronze.select([
    F.round(
        (F.sum(F.when(F.col(c).isNull(), 1).otherwise(0)) / F.lit(n) * 100),
        2   # <-- number of decimal places
    ).alias(c)
    for c in df_bronze.columns
])

display(missing.toPandas().T.reset_index()
        .rename(columns={"index":"column", 0:"percent_missing"}))
print("Column Level Missing Count:", missing.count())

#### 1.2.2 Duplicate checks (INCIDENT_NUMBER)

In [0]:
display(
    df_bronze.agg(
        F.count("*").alias("total_rows"),
        F.countDistinct("_id").alias("distinct_incident_number")
    )
)

dups = (df_bronze.groupBy("_id")
          .count()
          .filter(F.col("count") > 1)
          .orderBy(F.desc("count")))

display(dups)

In [0]:
if dups.count() > 0:
    sample_id = dups.select("_id").first()["_id"]
    display(df_bronze.filter(F.col("_id") == sample_id))

### 1.2.3 Quick profiling for future harmonization (top categories)

In [0]:
display(df_bronze.groupBy("Final_Incident_Type").count().orderBy(F.desc("count")).limit(30))
display(df_bronze.groupBy("Call_Source").count().orderBy(F.desc("count")).limit(30))
display(df_bronze.groupBy("Incident_Station_Area").count().orderBy(F.desc("count")).limit(30))
display(df_bronze.groupBy("Event_Alarm_Level").count().orderBy(F.desc("count")).limit(10))

## 2. Silver Layer Inspection

### 2.1 Load + schema sanity

In [0]:
df_silver = spark.read.table("workspace.capstone_project.tfs_incidents_silver")

print("SILVER row count:", df_silver.count())
df_silver.printSchema()
display(df_silver.limit(5))

### 2.2 Missing Values Check(%)

In [0]:
n = df_silver.count()

missing_pct = df_silver.select([
    F.round((F.sum(F.col(c).isNull().cast("int")) / F.lit(n)) * 100, 2).alias(c)
    for c in df_silver.columns
])

missing_pct_t = (
    missing_pct.selectExpr(
        "stack({}, {}) as (column, percent_missing)".format(
            len(missing_pct.columns),
            ",".join([f"'{c}', {c}" for c in missing_pct.columns])
        )
    )
    .orderBy(F.desc("percent_missing"))
)

display(missing_pct_t)


### 2.3 Timestamp Conversion Health Check

In [0]:
display(
    df_silver.select(
        F.sum(F.col("alarm_time").isNull().cast("int")).alias("null_alarm_time"),
        F.sum(F.col("arrival_time").isNull().cast("int")).alias("null_arrival_time"),
        F.sum(F.col("clear_time").isNull().cast("int")).alias("null_clear_time")
    )
)


### 2.4 Duplicate Checks
Check `_id` and `incident_numbner` as key.

In [0]:
display(
    df_silver.agg(
        F.count("*").alias("total_rows"),
        F.countDistinct("_id").alias("distinct__id"),
        F.countDistinct("INCIDENT_NUMBER").alias("distinct_incident_number")
    )
)

dups_id = df_silver.groupBy("_id").count().filter(F.col("count") > 1)
display(dups_id)

dups_key = (
    df_silver.groupBy("INCIDENT_NUMBER", "alarm_time")
             .count()
             .filter(F.col("count") > 1)
             .orderBy(F.desc("count"))
)
display(dups_key)


### 2.5 Response Time Sanity
The response time should be non-negative due to valid_timeline.

In [0]:
display(
    df_silver.agg(
        F.count("response_time_minutes").alias("n_non_null"),
        F.round(F.min("response_time_minutes"), 2).alias("min_min"),
        F.round(F.expr("percentile_approx(response_time_minutes, 0.5)"), 2).alias("p50_min"),
        F.round(F.expr("percentile_approx(response_time_minutes, 0.9)"), 2).alias("p90_min"),
        F.round(F.expr("percentile_approx(response_time_minutes, 0.95)"), 2).alias("p95_min"),
        F.round(F.max("response_time_minutes"), 2).alias("max_min")
    )
)

#### Check for Negative (should be 0)


In [0]:
display(
    df_silver.select(
        F.sum((F.col("response_time_seconds") < 0).cast("int")).alias("neg_response_seconds"),
        F.sum((F.col("incident_duration_minutes") < 0).cast("int")).alias("neg_incident_duration_min")
    )
)

### 2.6 Constraint Validation (alarm level, rescued)

In [0]:
display(
    df_silver.agg(
        F.min("Event_Alarm_Level").alias("min_alarm_level"),
        F.max("Event_Alarm_Level").alias("max_alarm_level"),
        F.min("Persons_Rescued").alias("min_rescued"),
        F.max("Persons_Rescued").alias("max_rescued")
    )
)

## 3. Gold Layer Inspection
Note that current Toronto Gold is more “analytics-enriched” (time_of_day, weekend, calls_past_30m/60m) but not yet harmonized to NYC’s final modeling schema. For ETL QA, we just validate correctness and sanity.

### 3.1 Data Loading and Sanity Check

In [0]:
df_gold = spark.read.table("workspace.capstone_project.tfs_incidents_gold")

print("GOLD row count:", df_gold.count())
df_gold.printSchema()
display(df_gold.limit(5))

### 3.2 Missing Values Check

In [0]:
n = df_gold.count()

missing_pct = df_gold.select([
    F.round((F.sum(F.col(c).isNull().cast("int")) / F.lit(n)) * 100, 2).alias(c)
    for c in df_gold.columns
])

missing_pct_t = (
    missing_pct.selectExpr(
        "stack({}, {}) as (column, percent_missing)".format(
            len(missing_pct.columns),
            ",".join([f"'{c}', {c}" for c in missing_pct.columns])
        )
    )
    .orderBy(F.desc("percent_missing"))
)

display(missing_pct_t)

### 3.3 Check demand feature sanity (no negatives)

In [0]:
display(
    df_gold.agg(
        F.min("calls_past_30m").alias("min_calls_30m"),
        F.max("calls_past_30m").alias("max_calls_30m"),
        F.min("calls_past_60m").alias("min_calls_60m"),
        F.max("calls_past_60m").alias("max_calls_60m")
    )
)

### 3.4 Validate time feature ranges

In [0]:
display(
    df_gold.agg(
        F.min("incident_hour").alias("min_hour"),
        F.max("incident_hour").alias("max_hour")
    )
)

display(df_gold.groupBy("time_of_day").count().orderBy(F.desc("count")))
display(df_gold.groupBy("is_weekend").count())
display(df_gold.groupBy("season").count().orderBy("season"))

### 3.5 Response time sanity still holds in Gold

In [0]:
display(
    df_gold.agg(
        F.count("response_time_minutes").alias("n_non_null"),
        F.round(F.min("response_time_minutes"), 2).alias("min_min"),
        F.round(F.expr("percentile_approx(response_time_minutes, 0.5)"), 2).alias("p50_min"),
        F.round(F.expr("percentile_approx(response_time_minutes, 0.9)"), 2).alias("p90_min"),
        F.round(F.expr("percentile_approx(response_time_minutes, 0.95)"), 2).alias("p95_min"),
        F.round(F.max("response_time_minutes"), 2).alias("max_min")
    )
)

### 3.6 Duplicate Check

In [0]:
display(
    df_gold.agg(
        F.count("*").alias("total_rows"),
        F.countDistinct("_id").alias("distinct__id")
    )
)

display(
    df_gold.groupBy("_id").count().filter(F.col("count") > 1).orderBy(F.desc("count"))
)